<a href="https://colab.research.google.com/github/tb-harris/neuroscience-2025/blob/main/9_Multiple_Regression_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import pandas as pd # pandas (DataFrames)
import seaborn as sns # graphs
import statsmodels.formula.api as smf # regression models
# installing and importing the custom diagnostics module
%pip install git+https://github.com/tb-harris/diagnostics.git
from diagnostics import LinearRegDiagnostic

  Cloning https://github.com/tb-harris/diagnostics.git to /tmp/pip-req-build-em6l32qc
  Running command git clone --filter=blob:none --quiet https://github.com/tb-harris/diagnostics.git /tmp/pip-req-build-em6l32qc
  Resolved https://github.com/tb-harris/diagnostics.git to commit 63e7ed899813d387d104118e3b35d435b398c0df
  Preparing metadata (setup.py) ... done
  Created wheel for diagnostics: filename=diagnostics-0.1-py3-none-any.whl size=4726 sha256=9b97ff6feaa2b64809a633b26f9b72a19e4dc8ac385e2df4bbd2586dfe6d31dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-qejfsni1/wheels/dc/6f/7f/5d7dbbdace3319b3bc68e42a4691a9f8e3605e00b9f8b8ce2b
Successfully built diagnostics


In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read in the data

In [ ]:
cell_metadata = pd.read_csv("/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/20200711_patchseq_metadata_mouse.csv", index_col="cell_specimen_id")

rna_data = pd.read_hdf("/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/gene_counts_ppm.hdf5.lz4")

morph_additional = pd.read_csv(
    "/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/morph_features_expanded.csv",
    index_col="cell_specimen_id"
)

morph_positional = pd.read_csv(
    "/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/morph_features_positional.csv",
    index_col="cell_specimen_id"
)

### Dataframe of genes and morphology together (only include full reconstruction and drop cell with NaN)

In [ ]:
morph_full_recon = morph_additional[cell_metadata["neuron_reconstruction_type"] == "full"]
gene_morph = morph_full_recon.join(rna_data)
gene_morph = gene_morph.dropna()

nonzero_genes = gene_morph.sum() != 0
gene_morph = gene_morph.loc[:, nonzero_genes]

/tmp/ipython-input-5-2565288093.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  morph_full_recon = morph_additional[cell_metadata["neuron_reconstruction_type"] == "full"]


In [ ]:
gene_only = gene_morph.iloc[:,159:]

gene_only.head()

,0610007P14Rik,0610009B22Rik,0610009E02Rik,0610009L18Rik,0610009O20Rik,0610010F05Rik,0610010K14Rik,0610011F06Rik,0610012G03Rik,0610025J13Rik,...,Zxda,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6
cell_specimen_id,,,,,,,,,,,,,,,,,,,,,
824909637,46.921360,0.998327,17.969882,18.968209,18.968209,59.899608,48.918013,0.0,36.938092,0.000000,...,0.000000,0.000000,0.000000,0.0,30.948131,16.971556,307.484656,28.951477,0.0,49.916340
869634031,0.000000,0.000000,13.415590,13.415590,1.916513,93.909130,47.912822,0.0,42.163283,0.000000,...,0.000000,24.914667,118.823798,0.0,126.489849,68.994463,153.321029,74.744002,0.0,55.578873
895740736,3.053482,39.695263,0.000000,13.231754,6.106963,227.993303,12.213927,0.0,17.303063,0.000000,...,12.213927,0.000000,75.319216,0.0,147.584950,117.050133,15.267409,70.230080,0.0,18.320890
898559010,0.000000,28.542586,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,127.690514,1.502241,...,0.000000,0.000000,6.008965,0.0,0.000000,0.000000,30.044827,0.000000,0.0,0.000000
899368360,44.048888,1.074363,0.000000,0.000000,147.187747,153.633926,59.089971,0.0,56.941245,0.000000,...,0.000000,0.000000,23.635989,0.0,32.230894,0.000000,220.244439,61.238698,0.0,10.743631
